In [269]:
#importar librerias
import pandas as pd
import numpy as np

In [270]:
#leer bases de datos
cronicos = pd.read_csv('databases/RETO_df_cronicos.csv',low_memory=False)
egresos= pd.read_csv('databases/RETO_df_egresos.csv')   
usuarios = pd.read_csv('databases/RETO_df_usuarios.csv',low_memory=False)

In [271]:
#mostrar cronicos, egresos y usuarios
list(cronicos.columns)



['YEAR',
 'MES',
 'Fecha diligenciamiento',
 'Hora',
 'Atención',
 'Ingreso',
 'Tipo',
 'NRODOC',
 'Peso',
 'Talla',
 'IMC',
 'Clasificación IMC',
 'Saturación de Oxígeno (%)',
 'Perímetro Muslo',
 'Perímetro Cintura',
 'Pliegue Triceps',
 'Pliegue Abdomen',
 'Pliegue Muslo',
 'Sumatoria Pliegues',
 'Presión Arterial Sistólica',
 'Presión Arterial Diastólica',
 'Frecuencia Cardíaca en reposo',
 'Auto-calificacion Nivel de Ejercicio',
 'Constantes',
 'METS -Índice Metabólico',
 'VO2 - Máxima Cantidad de Oxígeno',
 'Índice de Fragilidad Groningen',
 'Calificación (Índice de Fragilidad)',
 'Tiempo en Segundos (Apoyo Monopodal)',
 'Calificación (Apoyo Monopodal)',
 'Tiempo en Segundos (Recorrer 5 metros)',
 'Velocidad (m/s)',
 'Calificación Velocidad',
 'Test Findrisc',
 'Índice Tobillo/Brazo',
 'Observaciones',
 'Presión arterial registro medico',
 'Diabetes Mellitus',
 'Tipo diabetes mellitus',
 'Es insulinorequiriente',
 'Tiempo con el diagnóstico',
 'Glicemia',
 'Fecha glicemia',
 'Hem

In [272]:
list(usuarios.columns)

['MES',
 'YEAR',
 'NRODOC',
 'SEXO',
 'FECHA NACIMIENTO',
 'EDAD',
 'DEPARTAMENTO',
 'MUNICIPIO',
 'BARRIO',
 'FECHA INICIO AL PGP',
 'FECHA PRIMERA CLASE FUNCIONAL',
 'PRIMERA CLASE FUNCIONAL',
 'FECHA ÚLTIMA CLASE FUNCIONAL',
 'ÚLTIMA CLASE FUNCIONAL',
 'CICLO_VITAL',
 'QUINQUENIO']

In [273]:
egresos.columns

Index(['YEAR', 'MES', 'NRO ATENCION', 'NRO INGRESO', 'SERVICIO HABILITADO COD',
       'SERVICIO HABILITADO', 'TIPO IDENTIFICACION', 'NRODOC',
       'FECHA NACIMIENTO', 'REGIMEN AFILIACION', 'EPS VALIDADA',
       'MODALIDAD CONTRATO', 'FUENTE FINANCIACION1', 'FUENTE FINANCIACION2',
       'SERVICIO ADMITE', 'BLOQUE ANTERIOR', 'PISO ANTERIOR',
       'SERVICIO ANTERIOR', 'NRO CAMA ANTERIOR', 'VIA INGRESO', 'BLOQUE',
       'UNIDAD ESTRATEGICA', 'PISO', 'NRO CAMA', 'SERVICIO EGRESO',
       'TIPO EGRESO', 'FECHA INGRESO CLINICA', 'FECHA INGRESO SERVICIO',
       'FECHA POSIBLE ALTA', 'FECHA ALTA MEDICA', 'FECHA RECAUDO',
       'FECHA CAMILLERO', 'FECHA ENFERMERIA', 'FECHA FACTURACION AUDIFARMA',
       'FECHA FARMACIA', 'FECHA SALIDA', 'FECHA ASEO',
       'DEMORA ASIGNACION CAMA', 'DEMORA APLICACION MEDICAMENTO',
       'DEMORA SALIDA CLINICA (DIAS)', 'TRANSFUSION SANGRE', 'ANTIBIOTICO',
       'ALTA MEDICA', 'POSIBLE ALTA', 'DXPRINCIPAL EGRESO COD',
       'DXPRINCIPAL EGRESO', 'DX 

## filtro inicial 

In [274]:
#crear tabla nueva con los usuarios que tienen un ciclo vital en vejez
vejez = usuarios[usuarios["CICLO_VITAL"] == "Vejez"]
vejez.shape
#unir las 3 bases de datos
df1 = pd.merge(vejez, egresos, on='NRODOC', how='inner').merge(cronicos, on='NRODOC', how='inner')
#crear tabla nueva con la base de datos egresos con MODALIDAD CONTRATO = PGP
df1= df1[df1["MODALIDAD CONTRATO"] == "PGP"]
df1.shape

(549854, 370)

In [275]:
#convertir a formato "2017-04-30 00:00:00"  fecha FECHA INICIO AL PGP 
df1["FECHA INICIO AL PGP"] = pd.to_datetime(df1["FECHA INICIO AL PGP"], format='%Y-%m-%d %H:%M:%S')


#quiere ver el mayor y minimo de FECHA INICIO AL PGP
print(df1["FECHA INICIO AL PGP"].max())
print(df1["FECHA INICIO AL PGP"].min())

2022-02-03 00:00:00
2017-04-01 00:00:00


In [276]:
df1["NRODOC"].value_counts() # se puede observar que hay personas que tuvieron que ingresar hasta 3744 veces al PGP

NRODOC
308373    3744
307942    3360
307148    3136
309697    2592
306404    2304
          ... 
311672       3
313763       2
313282       2
312437       2
309859       2
Name: count, Length: 1830, dtype: int64

In [277]:
df1['FECHA INGRESO CLINICA'] = pd.to_datetime(df1['FECHA INGRESO CLINICA'])
df1['FECHA SALIDA'] = pd.to_datetime(df1['FECHA SALIDA'])

# Calcula la diferencia y crea una nueva columna para almacenarla
df1['TIEMPO DE ESTANCIA'] = df1['FECHA SALIDA'] - df1['FECHA INGRESO CLINICA']
#muestra la base de datos objetivo  con la nueva columna
df1.head() 



,MES_x,YEAR_x,NRODOC,SEXO,FECHA NACIMIENTO_x,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,FECHA INICIO AL PGP,FECHA PRIMERA CLASE FUNCIONAL,PRIMERA CLASE FUNCIONAL,FECHA ÚLTIMA CLASE FUNCIONAL,ÚLTIMA CLASE FUNCIONAL,CICLO_VITAL,QUINQUENIO,YEAR_y,MES_y,NRO ATENCION,NRO INGRESO,SERVICIO HABILITADO COD,SERVICIO HABILITADO,TIPO IDENTIFICACION,FECHA NACIMIENTO_y,REGIMEN AFILIACION,EPS VALIDADA,MODALIDAD CONTRATO,FUENTE FINANCIACION1,FUENTE FINANCIACION2,SERVICIO ADMITE,BLOQUE ANTERIOR,PISO ANTERIOR,SERVICIO ANTERIOR,NRO CAMA ANTERIOR,VIA INGRESO,BLOQUE,UNIDAD ESTRATEGICA,PISO,NRO CAMA,SERVICIO EGRESO,TIPO EGRESO,FECHA INGRESO CLINICA,FECHA INGRESO SERVICIO,FECHA POSIBLE ALTA,FECHA ALTA MEDICA,FECHA RECAUDO,FECHA CAMILLERO,FECHA ENFERMERIA,FECHA FACTURACION AUDIFARMA,FECHA FARMACIA,FECHA SALIDA,FECHA ASEO,DEMORA ASIGNACION CAMA,DEMORA APLICACION MEDICAMENTO,DEMORA SALIDA CLINICA (DIAS),TRANSFUSION SANGRE,ANTIBIOTICO,ALTA MEDICA,POSIBLE ALTA,DXPRINCIPAL EGRESO COD,DXPRINCIPAL EGRESO,DX RELACIONADO1 COD,DX RELACIONADO1,DX RELACIONADO2 COD,DX RELACIONADO2,DX RELACIONADO3 COD,DX RELACIONADO3,TIPO DIAGNOSTICO PRINCIPAL,PERTINENCIA DIAGNOSTICA,DX PRINCIPAL EGRESO CATEGORIA COD,DX PRINCIPAL EGRESO CATEGORIA,DX PRINCIPAL EGRESO CAPITULO COD,DX PRINCIPAL EGRESO CAPITULO,CAUSA BASICA MUERTE COD,CAUSA BASICA MUERTE,CAUSA BASICA CATEGORIA COD,CAUSA BASICA CATEGORIA,CAUSA BASICA CAPITULO COD,CAUSA BASICA CAPITULO,PROFESIONAL ESPECIALIDAD,PROFESIONAL ESPECIALIDAD GRD,YEAR,MES,Fecha diligenciamiento,Hora,Atención,Ingreso,Tipo,Peso,Talla,IMC,Clasificación IMC,Saturación de Oxígeno (%),Perímetro Muslo,Perímetro Cintura,Pliegue Triceps,Pliegue Abdomen,Pliegue Muslo,Sumatoria Pliegues,Presión Arterial Sistólica,Presión Arterial Diastólica,Frecuencia Cardíaca en reposo,Auto-calificacion Nivel de Ejercicio,Constantes,METS -Índice Metabólico,VO2 - Máxima Cantidad de Oxígeno,Índice de Fragilidad Groningen,Calificación (Índice de Fragilidad),Tiempo en Segundos (Apoyo Monopodal),Calificación (Apoyo Monopodal),Tiempo en Segundos (Recorrer 5 metros),Velocidad (m/s),Calificación Velocidad,Test Findrisc,Índice Tobillo/Brazo,Observaciones,Presión arterial registro medico,Diabetes Mellitus,Tipo diabetes mellitus,Es insulinorequiriente,Tiempo con el diagnóstico,Glicemia,Fecha glicemia,Hemoglobina Glicada,Fecha hemoglobina glicosilada,Control Diabetes,Tiene riesgo de tener Diabetes Mellitus,Tiene HTA,Control HTA,Tiene riesgo de tener HTA,Tiempo con el diagnóstico1,Tiene EPOC,Espirometria,VEF1/CVF,VEF1/VFC Posbroncodilatador,Gravedad,Diagnóstico EPOC,Disnea MMRC,Clasificación,CAT,Número de exacerbaciones último año (Que hayan necesitado hospitalizado),Clasificación GOLD,Clasificación1,Clasificación BODEX,Oxígeno dependiente,Tiene gases arteriales,Resultado,EPOC (Clasificación BODEX),Enfermedad Coronaria (En el último año),Insuficiencia Cardíaca,Valvulopatía,Arritmia o paciente con dispositivo,Sufre de alguna enfermedad cardiovascular,¿Cuál?,Tabaquismo,Cuantos cigarrillos día,Años de Consumo,Indice paquete/año,Lipoproteina,Fecha LDL,HDL,Fecha HDL,Colesterol total,Fecha colesterol total,Trigliceridos,Fecha trigliceridos,Clasificación de Framinghan,Creatinina 1 consulta,Fecha creatinina1,Tasa de Filtración Glomerular TFG,Estadio de la Enfermedad Renal,Microalbuminuria,Fecha microalbuminuria,Hormona Estimulante de la Tiroides (TSH),Otras Morbilidades,Clase Funcional,Creatinina 2 consulta,Fecha creatinina2,Tasa de Filtración Glomerular TFG2,Meses de diferencia entre TFG,Cambio de TFG,Clasificación Cambio de TFG,Última mamografía,Última citología,Última cita odontológica,Última cita con Optometría / Oftallmología,Último PSA,Basciloscopia,Fecha basciloscopia,Última vacuna de influenza,Fecha Vacuna Neumococo,Fecha vacuna neumococo veintitrés valentes,Úlcera de pie diabético,Remisión,Diagnóstico Principal,Análisis y conducta a seguir,Tiene próximo control,Fecha Próximo Control,Tipo control,Requiere cita de morbilidad,Fecha cita morbilidad,Ambito según el médico,Diagnostico1,NombreDiag

In [278]:
# los valores unicos de la columna  'YEAR' de la base de datos   
df1['YEAR'].unique()

array([2017, 2018, 2021, 2022], dtype=int64)

In [279]:
# eliminar de la base de datos df1 los valores de la columna 'YEAR' 2017 y 2018    
df1=df1[df1["YEAR"] != 2017]
df1=df1[df1["YEAR"] != 2018]

In [280]:
# muestrame un listados de las 10 que mas se repitan en la columna "DX PRINCIPAL EGRESO CAPITULO" de la base de datos df1 
df1["DX PRINCIPAL EGRESO CAPITULO"].value_counts().head(10)

DX PRINCIPAL EGRESO CAPITULO
ENFERMEDADES DEL SISTEMA RESPIRATORIO                                             80804
ENFERMEDADES DEL SISTEMA CIRCULATORIO                                             58506
ENFERMEDADES DEL SISTEMA DIGESTIVO                                                44178
ENFERMEDADES DEL SISTEMA GENITOURINARIO                                           40311
TRAUMATISMOS, ENVENENAMIENTO Y ALGUNAS OTRAS CONSECUENCIAS DE CAUSAS EXTERNAS     32499
ENFERMEDADES ENDOCRINAS, NUTRICIONALES Y METABOLICAS                              30482
ENFERMEDADES DEL SISTEMA NERVIOSO                                                 18354
CODIGOS PARA PROPOSITOS ESPECIALES                                                15056
CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIAS                                    7433
TUMORES [NEOPLASIAS]                                                               7329
Name: count, dtype: int64

In [281]:
#MUESTRAME EL NUMERO DE VARIABLES QUE TIENE LA COLUMNAS "DX PRINCIPAL EGRESO CAPITULO" DE LA BASE DE DATOS DF1
df1["DX PRINCIPAL EGRESO CAPITULO"].nunique()
 # SE OBSERVA QUE CON  LAS PERIMERAS 8 VARIABLES SE TIENE EL 89.33% APROXIMADAMENTE DE LA INFORMACION DE LA BASE DE DATOS DF1 RELACIONADA CON LA COLUMNA "DX PRINCIPAL EGRESO CAPITULO"   
df1["DX PRINCIPAL EGRESO CAPITULO"].value_counts().head(8).sum()/df1["DX PRINCIPAL EGRESO CAPITULO"].value_counts().sum()*100  


89.33748134094111

In [282]:
#eliminar las filas de las columnas "DX PRINCIPAL EGRESO CAPITULO" que no esten en las 8 primeras variables
df1=df1[df1["DX PRINCIPAL EGRESO CAPITULO"].isin(df1["DX PRINCIPAL EGRESO CAPITULO"].value_counts().head(8).index)]

In [283]:
# ver columnas nulas mayores a 0 en la base de datos df1 en porcentaje 
nulos=df1.isnull().sum()/len(df1)*100
# ver los nulos vayores a 80 de la base nulos
nulos[nulos>80]
# y eliminarlos de la base de datos df1
df1=df1.drop(columns=nulos[nulos>80].index)

In [284]:
#ver la base de datos df1
df1.head(1)

,MES_x,YEAR_x,NRODOC,SEXO,FECHA NACIMIENTO_x,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,FECHA INICIO AL PGP,FECHA PRIMERA CLASE FUNCIONAL,PRIMERA CLASE FUNCIONAL,FECHA ÚLTIMA CLASE FUNCIONAL,ÚLTIMA CLASE FUNCIONAL,CICLO_VITAL,QUINQUENIO,YEAR_y,MES_y,NRO ATENCION,NRO INGRESO,SERVICIO HABILITADO COD,SERVICIO HABILITADO,TIPO IDENTIFICACION,FECHA NACIMIENTO_y,REGIMEN AFILIACION,EPS VALIDADA,MODALIDAD CONTRATO,FUENTE FINANCIACION1,FUENTE FINANCIACION2,SERVICIO ADMITE,BLOQUE ANTERIOR,PISO ANTERIOR,SERVICIO ANTERIOR,NRO CAMA ANTERIOR,VIA INGRESO,BLOQUE,UNIDAD ESTRATEGICA,PISO,NRO CAMA,SERVICIO EGRESO,TIPO EGRESO,FECHA INGRESO CLINICA,FECHA INGRESO SERVICIO,FECHA ALTA MEDICA,FECHA RECAUDO,FECHA CAMILLERO,FECHA ENFERMERIA,FECHA FACTURACION AUDIFARMA,FECHA FARMACIA,FECHA SALIDA,FECHA ASEO,DEMORA ASIGNACION CAMA,DEMORA APLICACION MEDICAMENTO,DEMORA SALIDA CLINICA (DIAS),TRANSFUSION SANGRE,ANTIBIOTICO,ALTA MEDICA,POSIBLE ALTA,DXPRINCIPAL EGRESO COD,DXPRINCIPAL EGRESO,DX RELACIONADO1 COD,DX RELACIONADO1,DX RELACIONADO2 COD,DX RELACIONADO2,DX RELACIONADO3 COD,DX RELACIONADO3,TIPO DIAGNOSTICO PRINCIPAL,PERTINENCIA DIAGNOSTICA,DX PRINCIPAL EGRESO CATEGORIA COD,DX PRINCIPAL EGRESO CATEGORIA,DX PRINCIPAL EGRESO CAPITULO COD,DX PRINCIPAL EGRESO CAPITULO,CAUSA BASICA MUERTE,CAUSA BASICA CATEGORIA,CAUSA BASICA CAPITULO,PROFESIONAL ESPECIALIDAD,PROFESIONAL ESPECIALIDAD GRD,YEAR,MES,Fecha diligenciamiento,Hora,Atención,Ingreso,Tipo,Peso,Talla,IMC,Clasificación IMC,Saturación de Oxígeno (%),Perímetro Muslo,Perímetro Cintura,Pliegue Triceps,Pliegue Abdomen,Pliegue Muslo,Sumatoria Pliegues,Presión Arterial Sistólica,Presión Arterial Diastólica,Frecuencia Cardíaca en reposo,Auto-calificacion Nivel de Ejercicio,Constantes,METS -Índice Metabólico,VO2 - Máxima Cantidad de Oxígeno,Índice de Fragilidad Groningen,Calificación (Índice de Fragilidad),Tiempo en Segundos (Apoyo Monopodal),Calificación (Apoyo Monopodal),Tiempo en Segundos (Recorrer 5 metros),Velocidad (m/s),Calificación Velocidad,Test Findrisc,Índice Tobillo/Brazo,Observaciones,Presión arterial registro medico,Diabetes Mellitus,Tipo diabetes mellitus,Tiempo con el diagnóstico,Glicemia,Fecha glicemia,Hemoglobina Glicada,Fecha hemoglobina glicosilada,Control Diabetes,Tiene HTA,Control HTA,Tiene riesgo de tener HTA,Tiempo con el diagnóstico1,Tiene EPOC,EPOC (Clasificación BODEX),Enfermedad Coronaria (En el último año),Insuficiencia Cardíaca,Valvulopatía,Arritmia o paciente con dispositivo,Sufre de alguna enfermedad cardiovascular,¿Cuál?,Tabaquismo,Cuantos cigarrillos día,Años de Consumo,Lipoproteina,Fecha LDL,HDL,Fecha HDL,Colesterol total,Fecha colesterol total,Trigliceridos,Fecha trigliceridos,Clasificación de Framinghan,Creatinina 1 consulta,Fecha creatinina1,Tasa de Filtración Glomerular TFG,Estadio de la Enfermedad Renal,Microalbuminuria,Fecha microalbuminuria,Hormona Estimulante de la Tiroides (TSH),Clase Funcional,Creatinina 2 consulta,Fecha creatinina2,Tasa de Filtración Glomerular TFG2,Meses de diferencia entre TFG,Cambio de TFG,Clasificación Cambio de TFG,Basciloscopia,Fecha basciloscopia,Úlcera de pie diabético,Remisión,Diagnóstico Principal,Análisis y conducta a seguir,Tiene próximo control,Fecha Próximo Control,Tipo control,Requiere cita de morbilidad,Fecha cita morbilidad,Ambito según el médico,Diagnostico1,NombreDiagnostico1,Diagnostico2,NombreDiagnostico2,Diagnostico3,NombreDiagnostico3,Diagnostico4,NombreDiagnostico4,Diagnostico5,NombreDiagnostico5,TIEMPO DE ESTANCIA
5,SEPTIEMBRE,2017,310364,M,NaN,66.0,NaN,NaN,MEDELLIN,NaT,NaN,NaN,NaN,NaN,Vejez,65-69,2018,JUNIO,4052683,2272079,101.0,General adultos,CC,1949-05-23 00:00:00.000,Contributivo,NUEVA EPS S.A. PGP,PGP,NUEVA EPS,NUEVA EPS,Hospitalización,Sin Informacion,Sin Informacion,Sin Informacion,Sin Informacion,Remitido,Bloque 1,Hospitalizacion Adultos,Piso 7,727AN,Piso 7 Medicina Interna Norte,Alta Médica,2018-06-19 19:37:21.920,2018-06-19 19:37:21.920,2018-06-26 16:39:59.950,NaN,NaN,NaN,NaN,NaN,2018-06-26 18:26:56,NaN,00:00,00:00,0.0,NO,SI,Si

In [285]:
# crear una base de datos nueva llamada df2 con las siguientes columnas NRODOC,SEXO, EDAD, DEPARTAMENTO,	MUNICIPIO,	BARRIO, ÚLTIMA CLASE FUNCIONAL, SERVICIO HABILITADO, TIPO IDENTIFICACION, REGIMEN AFILIACION,  EPS VALIDADA, SERVICIO ADMITE, BLOQUE ANTERIOR,	PISO ANTERIOR,	SERVICIO ANTERIOR,	NRO CAMA ANTERIOR, VIA INGRESO, BLOQUE,	UNIDAD ESTRATEGICA,	PISO,	NRO CAMA,	SERVICIO EGRESO,	TIPO EGRESO,	FECHA INGRESO CLINICA, FECHA SALIDA,TRANSFUSION SANGRE,	ANTIBIOTICO, DX PRINCIPAL EGRESO CAPITULO, PROFESIONAL ESPECIALIDAD GRD, Peso	,Talla	,IMC	,Clasificación IMC	,Saturación de Oxígeno (%)	,Perímetro Muslo	,Perímetro Cintura	,Pliegue Triceps	,Pliegue Abdomen	, Pliegue Muslo,	Sumatoria Pliegues , Presión Arterial Sistólica,	Presión Arterial Diastólica	, Frecuencia Cardíaca en reposo	, Auto-calificacion Nivel de Ejercicio	, Constantes	, METS -Índice Metabólico	, VO2 - Máxima Cantidad de Oxígeno	, Índice de Fragilidad Groningen	,Calificación (Índice de Fragilidad)	,Tiempo en Segundos (Apoyo Monopodal) , Calificación (Apoyo Monopodal)	, Tiempo en Segundos (Recorrer 5 metros)	, Velocidad (m/s)	, Calificación Velocidad	, Test Findrisc	, Índice Tobillo/Brazo	, Presión arterial registro medico	, Diabetes Mellitus	, Tipo diabetes mellitus	, Tiempo con el diagnóstico	, Glicemia	, Fecha glicemia	, Hemoglobina Glicada	, Fecha hemoglobina glicosilada	, Control Diabetes	, Tiene HTA	, Control HTA	, Tiene riesgo de tener HTA	, Tiempo con el diagnóstico1	, Tiene EPOC	, EPOC (Clasificación BODEX)	, Enfermedad Coronaria (En el último año)	, Insuficiencia Cardíaca	, Valvulopatía	, Arritmia o paciente con dispositivo	, Sufre de alguna enfermedad cardiovascular	, ¿Cuál?	, Tabaquismo	, Cuantos cigarrillos día	, Años de Consumo	, Lipoproteina	, Fecha LDL	, HDL	, Fecha HDL	, Colesterol total	, Fecha colesterol total	, Trigliceridos	, Fecha trigliceridos	, Clasificación de Framinghan	, Creatinina 1 consulta	, Fecha creatinina1	, Tasa de Filtración Glomerular TFG	, Estadio de la Enfermedad Renal	, Microalbuminuria	, Fecha microalbuminuria	, Hormona Estimulante de la Tiroides (TSH)	, Clase Funcional	, Creatinina 2 consulta	, Fecha creatinina2	, Tasa de Filtración Glomerular TFG2	, Meses de diferencia entre TFG	, Cambio de TFG	, Clasificación Cambio de TFG	Basciloscopia	, Úlcera de pie diabético	Remisión	, Diagnóstico Principal	, Análisis y conducta a seguir	 , Tiene próximo control	, Fecha Próximo Control	  , Tipo control	, Requiere cita de morbilidad	Fecha cita morbilidad	, Ambito según el médico	, Diagnostico1, 	NombreDiagnostico1	, Diagnostico2	 , NombreDiagnostico2	, Diagnostico3	, NombreDiagnostico3	, TIEMPO DE ESTANCIA , de la base de datos df1
df2=df1[["NRODOC","SEXO","EDAD","DEPARTAMENTO","MUNICIPIO","BARRIO","ÚLTIMA CLASE FUNCIONAL","SERVICIO HABILITADO","TIPO IDENTIFICACION","REGIMEN AFILIACION","EPS VALIDADA","SERVICIO ADMITE","BLOQUE ANTERIOR","PISO ANTERIOR","SERVICIO ANTERIOR","NRO CAMA ANTERIOR","VIA INGRESO","BLOQUE","UNIDAD ESTRATEGICA","PISO","NRO CAMA","SERVICIO EGRESO","TIPO EGRESO","FECHA INGRESO CLINICA","FECHA SALIDA","TRANSFUSION SANGRE","ANTIBIOTICO","DX PRINCIPAL EGRESO CAPITULO","PROFESIONAL ESPECIALIDAD GRD","Peso","Talla","IMC","Clasificación IMC","Saturación de Oxígeno (%)","Perímetro Muslo","Perímetro Cintura","Pliegue Triceps","Pliegue Abdomen","Pliegue Muslo","Sumatoria Pliegues","Presión Arterial Sistólica","Presión Arterial Diastólica","Frecuencia Cardíaca en reposo","Auto-calificacion Nivel de Ejercicio","Constantes","METS -Índice Metabólico","VO2 - Máxima Cantidad de Oxígeno","Índice de Fragilidad Groningen","Calificación (Índice de Fragilidad)","Tiempo en Segundos (Apoyo Monopodal)","Calificación (Apoyo Monopodal)","Tiempo en Segundos (Recorrer 5 metros)","Velocidad (m/s)","Calificación Velocidad","Test Findrisc","Índice Tobillo/Brazo","Presión arterial registro medico","Diabetes Mellitus","Tipo diabetes mellitus","Tiempo con el diagnóstico","Glicemia","Fecha glicemia","Hemoglobina Glicada","Fecha hemoglobina glicosilada","Control Diabetes","Tiene HTA","Control HTA","Tiene riesgo de tener HTA","Tiempo con el diagnóstico1","Tiene EPOC","EPOC (Clasificación BODEX)","Enfermedad Coronaria (En el último año)", "Insuficiencia Cardíaca","Valvulopatía","Arritmia o paciente con dispositivo","Sufre de alguna enfermedad cardiovascular","¿Cuál?","Tabaquismo","Cuantos cigarrillos día","Años de Consumo","Lipoproteina","Fecha LDL","HDL","Fecha HDL","Colesterol total","Fecha colesterol total","Trigliceridos","Fecha trigliceridos","Clasificación de Framinghan","Creatinina 1 consulta","Fecha creatinina1","Tasa de Filtración Glomerular TFG","Estadio de la Enfermedad Renal","Microalbuminuria","Fecha microalbuminuria","Hormona Estimulante de la Tiroides (TSH)","Clase Funcional","Creatinina 2 consulta","Fecha creatinina2","Tasa de Filtración Glomerular TFG2","Meses de diferencia entre TFG","Cambio de TFG","Clasificación Cambio de TFG","Basciloscopia","Úlcera de pie diabético","Remisión","Diagnóstico Principal","Análisis y conducta a seguir","Tiene próximo control","Fecha Próximo Control","Tipo control","Requiere cita de morbilidad","Fecha cita morbilidad","Ambito según el médico","Diagnostico1","NombreDiagnostico1","Diagnostico2","NombreDiagnostico2","Diagnostico3","NombreDiagnostico3","TIEMPO DE ESTANCIA"]]
#ver la base de datos df2
df2.head(1)                                                                                 

,NRODOC,SEXO,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,ÚLTIMA CLASE FUNCIONAL,SERVICIO HABILITADO,TIPO IDENTIFICACION,REGIMEN AFILIACION,EPS VALIDADA,SERVICIO ADMITE,BLOQUE ANTERIOR,PISO ANTERIOR,SERVICIO ANTERIOR,NRO CAMA ANTERIOR,VIA INGRESO,BLOQUE,UNIDAD ESTRATEGICA,PISO,NRO CAMA,SERVICIO EGRESO,TIPO EGRESO,FECHA INGRESO CLINICA,FECHA SALIDA,TRANSFUSION SANGRE,ANTIBIOTICO,DX PRINCIPAL EGRESO CAPITULO,PROFESIONAL ESPECIALIDAD GRD,Peso,Talla,IMC,Clasificación IMC,Saturación de Oxígeno (%),Perímetro Muslo,Perímetro Cintura,Pliegue Triceps,Pliegue Abdomen,Pliegue Muslo,Sumatoria Pliegues,Presión Arterial Sistólica,Presión Arterial Diastólica,Frecuencia Cardíaca en reposo,Auto-calificacion Nivel de Ejercicio,Constantes,METS -Índice Metabólico,VO2 - Máxima Cantidad de Oxígeno,Índice de Fragilidad Groningen,Calificación (Índice de Fragilidad),Tiempo en Segundos (Apoyo Monopodal),Calificación (Apoyo Monopodal),Tiempo en Segundos (Recorrer 5 metros),Velocidad (m/s),Calificación Velocidad,Test Findrisc,Índice Tobillo/Brazo,Presión arterial registro medico,Diabetes Mellitus,Tipo diabetes mellitus,Tiempo con el diagnóstico,Glicemia,Fecha glicemia,Hemoglobina Glicada,Fecha hemoglobina glicosilada,Control Diabetes,Tiene HTA,Control HTA,Tiene riesgo de tener HTA,Tiempo con el diagnóstico1,Tiene EPOC,EPOC (Clasificación BODEX),Enfermedad Coronaria (En el último año),Insuficiencia Cardíaca,Valvulopatía,Arritmia o paciente con dispositivo,Sufre de alguna enfermedad cardiovascular,¿Cuál?,Tabaquismo,Cuantos cigarrillos día,Años de Consumo,Lipoproteina,Fecha LDL,HDL,Fecha HDL,Colesterol total,Fecha colesterol total,Trigliceridos,Fecha trigliceridos,Clasificación de Framinghan,Creatinina 1 consulta,Fecha creatinina1,Tasa de Filtración Glomerular TFG,Estadio de la Enfermedad Renal,Microalbuminuria,Fecha microalbuminuria,Hormona Estimulante de la Tiroides (TSH),Clase Funcional,Creatinina 2 consulta,Fecha creatinina2,Tasa de Filtración Glomerular TFG2,Meses de diferencia entre TFG,Cambio de TFG,Clasificación Cambio de TFG,Basciloscopia,Úlcera de pie diabético,Remisión,Diagnóstico Principal,Análisis y conducta a seguir,Tiene próximo control,Fecha Próximo Control,Tipo control,Requiere cita de morbilidad,Fecha cita morbilidad,Ambito según el médico,Diagnostico1,NombreDiagnostico1,Diagnostico2,NombreDiagnostico2,Diagnostico3,NombreDiagnostico3,TIEMPO DE ESTANCIA
5,310364,M,66.0,NaN,NaN,MEDELLIN,NaN,General adultos,CC,Contributivo,NUEVA EPS S.A. PGP,Hospitalización,Sin Informacion,Sin Informacion,Sin Informacion,Sin Informacion,Remitido,Bloque 1,Hospitalizacion Adultos,Piso 7,727AN,Piso 7 Medicina Interna Norte,Alta Médica,2018-06-19 19:37:21.920,2018-06-26 18:26:56,NO,SI,ENFERMEDADES DEL SISTEMA RESPIRATORIO,Medicina Interna,66.0,154.0,27.83,Sobrepeso,96.0,56.0,106.0,19.0,33.0,23.0,75.0,122,68,66,1,0.0,6.9289,24.2512,1.0,Normal,6,Malo - Fragil,5,1.0,Aceptable,NaN,0.91 - 1.30 Normal,NaN,No,NaN,NaN,92.0,2021-06-24 00:00:00,5.86,2021-06-24 00:00:00,No aplica,Si,Controlada,No aplica,21,Si,Leve,No aplica,No aplica,No,No,Si,HTA,No,0,0.0,0.0,NaN,49.4,2021-06-24 00:00:00,121.8,2021-06-24 00:00:00,102.5,2021-06-24 00:00:00,Bajo riesgo,0.77,2021-06-24 00:00:00,80.95,Estadio 2,3.57,2021-06-24 00:00:00,0.0,Clase funcional 2A,0.93,2019-08-20 00:00:00,67.03,10.0,1.39,Deterioro,No aplica,No,Grupal 2,J449 - ENFERMEDAD PULMONAR OBSTRUCTIVA CRONICA...,RECOMENDACIONES: Unos pocos gestos sencillos l...,Si,2021-11-25,Médico,No,NaN,Ambulatorio,I10X,HIPERTENSION ESENCIAL (PRIMARIA),E780,HIPERCOLESTEROLEMIA PURA,E669,OBESIDAD NO ESPECIFICADA,6 days 22:49:34.080000


In [286]:
#dimensiones de la base de datos df2
df2.shape

(320190, 121)

In [287]:
# mostrar las 15 columnas con mayor datos nulos de la base de datos df2
nulos=df2.isnull().sum()/len(df2)*100
nulos.sort_values(ascending=False).head(15)

Clasificación Cambio de TFG         72.814579
Fecha cita morbilidad               65.072613
NombreDiagnostico3                  58.738874
Diagnostico3                        58.738874
Test Findrisc                       53.360817
Tiempo con el diagnóstico           51.647772
EPOC (Clasificación BODEX)          50.614635
NombreDiagnostico2                  50.499079
Diagnostico2                        50.499079
Fecha LDL                           49.227334
¿Cuál?                              47.108904
Tipo diabetes mellitus              46.117930
Presión arterial registro medico    41.854212
MUNICIPIO                           39.648334
DEPARTAMENTO                        39.597739
dtype: float64

In [288]:
#eliminar las columnas (Fecha cita morbilidad, NombreDiagnostico3, Diagnostico3, Tiempo con el diagnóstico)de la base de datos df2
df2=df2.drop(columns=["Fecha creatinina1", "Diagnóstico Principal","Diagnostico1","Fecha creatinina2","Tiempo con el diagnóstico1","Fecha microalbuminuria", "Fecha hemoglobina glicosilada","Tipo control","Fecha Próximo Control","Fecha trigliceridos",             
"Fecha colesterol total","Fecha HDL","Fecha glicemia","Presión arterial registro medico","NombreDiagnostico1","Diagnostico2","NombreDiagnostico2","Fecha LDL","¿Cuál?","Fecha cita morbilidad","NombreDiagnostico3","Diagnostico3","Tiempo con el diagnóstico","Diagnostico2","NombreDiagnostico2","Fecha LDL","¿Cuál?"]) 


In [289]:
# modificar los nulos de las columnas (BLOQUE ANTERIOR,	PISO ANTERIOR,	SERVICIO ANTERIOR,	NRO CAMA ANTERIOR), con 0 en la base de datos df2
df2["BLOQUE ANTERIOR"]=df2["BLOQUE ANTERIOR"].fillna(0) #posiblemeente se trata de unpasiente que no a ingresado antes
df2["PISO ANTERIOR"]=df2["PISO ANTERIOR"].fillna(0)   #posiblemeente se trata de unpasiente que no a ingresado antes 
df2["SERVICIO ANTERIOR"]=df2["SERVICIO ANTERIOR"].fillna(0) #posiblemeente se trata de unpasiente que no a ingresado antes
df2["NRO CAMA ANTERIOR"]=df2["NRO CAMA ANTERIOR"].fillna(0)  #posiblemeente se trata de unpasiente que no a ingresado antes
df2["Índice Tobillo/Brazo"]=df2["Índice Tobillo/Brazo"].fillna(0)  #posiblemeente se trata de unpasiente que no a ingresado antes
df2["Clasificación Cambio de TFG"]=df2["Clasificación Cambio de TFG"].fillna("No aplica")  # posiblemnete la nulidad se debe a que no se le realizo la clasificacion
df2["Test Findrisc"]=df2["Test Findrisc"].fillna("No aplica") # posiblemnete la nulidad se debe a que no se le realizo la clasificacion
df2["EPOC (Clasificación BODEX)"]=df2["EPOC (Clasificación BODEX)"].fillna("No aplica") # posiblemnete la nulidad se debe a que no se le realizo la clasificacion
df2["Tipo diabetes mellitus"]=df2["Tipo diabetes mellitus"].fillna("No aplica") # posiblemnete la nulidad se debe a que no se le realizo la clasificacion
df2["MUNICIPIO"]=df2["MUNICIPIO"].fillna("Sin Dato") # se decide reemplazar la nulidad por sin dato
df2["DEPARTAMENTO"]=df2["DEPARTAMENTO"].fillna("Sin Dato") # se decide reemplazar la nulidad por sin dato
df2["Basciloscopia"]=df2["Basciloscopia"].fillna("No aplica") # posiblemnete la nulidad se debe a que no se le realizo la clasificacion
df2["BARRIO"]=df2["BARRIO"].fillna("Sin Dato") # se decide reemplazar la nulidad por sin dato
df2["ÚLTIMA CLASE FUNCIONAL"]=df2["ÚLTIMA CLASE FUNCIONAL"].fillna("Sin Dato") # se decide reemplazar la nulidad por sin dato
df2["ÚLTIMA CLASE FUNCIONAL"]=df2["ÚLTIMA CLASE FUNCIONAL"].replace(["Sin Clasificar","SIN CLASIFICAR"],"Sin Dato") # se decide reemplazar la Sin Clasificar, SIN CLASIFICAR por sin dato
df2["Estadio de la Enfermedad Renal"]=df2["Estadio de la Enfermedad Renal"].fillna("Estadio 0") # se decide reemplazar la nulidad por Estadio 0 ya que no requiere tratamiento
df2["Tiene próximo control"]=df2["Tiene próximo control"].fillna("No") # se decide reemplazar la nulidad por No ya que se asume que no tiene proximo control
df2["Análisis y conducta a seguir"]=df2["Análisis y conducta a seguir"].fillna("Sin dato") # se decide reemplazar la nulidad por sin dato
df2["HDL"]=df2["HDL"].fillna(40) # se decide reemplazar la nulidad por el valor 40 el cual representa epara las personas mayores de 20 años un nivel aceptable
df2["Trigliceridos"]=df2["Trigliceridos"].fillna(121.4) # se reemplaza la nulidad por 121.4 la cual es la mediana de los datos 
df2["Tiene EPOC"]=df2["Tiene EPOC"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Tiene riesgo de tener HTA"]=df2["Tiene riesgo de tener HTA"].fillna("No aplica") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Tiene HTA"]=df2["Tiene HTA"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Control Diabetes"]=df2["Control Diabetes"].fillna("No aplica") # se reemplaza la nulidad por No aplica ya que posiblemnete no aplica 
df2["Valvulopatía"]=df2["Valvulopatía"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Insuficiencia Cardíaca"]=df2["Insuficiencia Cardíaca"].fillna("No aplica") # se reemplaza la nulidad por No aplica ya que posiblemnete no aplica 
df2["Insuficiencia Cardíaca"]=df2["Insuficiencia Cardíaca"].replace(["No"],"No aplica")# se reemplaza el No por No aplica  
df2["Arritmia o paciente con dispositivo"]=df2["Arritmia o paciente con dispositivo"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Control HTA"]=df2["Control HTA"].fillna("No aplica") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Colesterol total"]=df2["Colesterol total"].fillna(133.1) # se reemplaza la nulidad por 29.55 la cual es la mediana de los datos 
df2["Creatinina 1 consulta"]=df2["Creatinina 1 consulta"].fillna(0.88) # se reemplaza la nulidad por 0.88 la cual es la mediana de los datos 
df2["Arritmia o paciente con dispositivo"]=df2["Arritmia o paciente con dispositivo"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Lipoproteina"]=df2["Lipoproteina"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Diabetes Mellitus"]=df2["Diabetes Mellitus"].fillna(29.55) # se reemplaza la nulidad por 29.55 la cual es la mediana de los datos 
df2["Tabaquismo"]=df2["Tabaquismo"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Sufre de alguna enfermedad cardiovascular"]=df2["Sufre de alguna enfermedad cardiovascular"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no aplica 
df2["Enfermedad Coronaria (En el último año)"]=df2["Enfermedad Coronaria (En el último año)"].fillna("No aplica") # se reemplaza la nulidad por No aplica ya que posiblemnete no aplica 
df2["Enfermedad Coronaria (En el último año)"]=df2["Enfermedad Coronaria (En el último año)"].replace(["No"],"No aplica")# se reemplaza el No por No aplica  
df2["Cuantos cigarrillos día"]=df2["Cuantos cigarrillos día"].fillna(0) # se reemplaza la nulidad por 0 la cual es la mediana de los datos 
df2["Úlcera de pie diabético"]=df2["Úlcera de pie diabético"].fillna("No") # se reemplaza la nulidad por No ya que posiblemnete no tiene ulcera de pie diabetico

In [290]:
#eliminar los datos nulos de la base de datos df2
df2=df2.dropna()


In [291]:
#dimensiones de la base de datos df2
df2.shape

(313052, 98)

In [292]:
# mostrar las 15 columnas con mayor datos nulos de la base de datos df2
nulos=df2.isnull().sum()/len(df2)*100
nulos.sort_values(ascending=False).head(15)

NRODOC                                       0.0
Años de Consumo                              0.0
Tabaquismo                                   0.0
Sufre de alguna enfermedad cardiovascular    0.0
Arritmia o paciente con dispositivo          0.0
Valvulopatía                                 0.0
Insuficiencia Cardíaca                       0.0
Enfermedad Coronaria (En el último año)      0.0
EPOC (Clasificación BODEX)                   0.0
Tiene EPOC                                   0.0
Tiene riesgo de tener HTA                    0.0
Control HTA                                  0.0
Tiene HTA                                    0.0
Control Diabetes                             0.0
Hemoglobina Glicada                          0.0
dtype: float64